In [1]:
!pip install pulp
from pulp import *

In [2]:
nodes = ['NEW', 'BOS', 'COL', 'RIC', 'ATL', 'MOB', 'JAC']

supply = {
    'NEW' : 200, 'BOS' : 0, 'COL' : 0, 'RIC' : 0 , 'ATL' : 0, 'MOB' : 0, 'JAC' : 300
}

demand = {
    'NEW' : 0, 'BOS' : 100, 'COL' : 60, 'RIC' : 80 , 'ATL' : 170, 'MOB' : 70, 'JAC' : 0
}

cost = {
    ('NEW','BOS'): 30,
    ('BOS','COL'): 50,
    ('COL','ATL'): 35,
    ('ATL','COL'): 40,
    ('NEW','RIC'): 40,
    ('ATL','RIC'): 30,
    ('ATL','MOB'): 35,
    ('MOB','ATL'): 25,
    ('JAC','MOB'): 50,
    ('JAC','ATL'): 45,
    ('JAC','RIC'): 50
}

links = list(cost.keys())

links[1][1]

'COL'

In [12]:
for (i,j) in links:
    print(i,'-',j)

NEW - BOS
BOS - COL
COL - ATL
ATL - COL
NEW - RIC
ATL - RIC
ATL - MOB
MOB - ATL
JAC - MOB
JAC - ATL
JAC - RIC


In [3]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Transshipment Problem", LpMinimize)

# A dictionary of the output variables
x = LpVariable.dicts("x",links,0)

# Create objective function
prob += lpSum([x[i]*cost[i] for i in links])

for n in nodes:
    prob += (supply[n]+ lpSum([x[(i,j)] for (i,j) in links if j == n]) >= demand[n]+ lpSum([x[(i,j)] for (i,j) in links if i == n])), "Flow Conservation in Node:%s"%n

In [4]:
prob

Transshipment Problem:
MINIMIZE
40*x_('ATL',_'COL') + 35*x_('ATL',_'MOB') + 30*x_('ATL',_'RIC') + 50*x_('BOS',_'COL') + 35*x_('COL',_'ATL') + 45*x_('JAC',_'ATL') + 50*x_('JAC',_'MOB') + 50*x_('JAC',_'RIC') + 25*x_('MOB',_'ATL') + 30*x_('NEW',_'BOS') + 40*x_('NEW',_'RIC') + 0
SUBJECT TO
Flow_Conservation_in_Node:NEW: - x_('NEW',_'BOS') - x_('NEW',_'RIC') >= -200

Flow_Conservation_in_Node:BOS: - x_('BOS',_'COL') + x_('NEW',_'BOS') >= 100

Flow_Conservation_in_Node:COL: x_('ATL',_'COL') + x_('BOS',_'COL')
 - x_('COL',_'ATL') >= 60

Flow_Conservation_in_Node:RIC: x_('ATL',_'RIC') + x_('JAC',_'RIC')
 + x_('NEW',_'RIC') >= 80

Flow_Conservation_in_Node:ATL: - x_('ATL',_'COL') - x_('ATL',_'MOB')
 - x_('ATL',_'RIC') + x_('COL',_'ATL') + x_('JAC',_'ATL') + x_('MOB',_'ATL')
 >= 170

Flow_Conservation_in_Node:MOB: x_('ATL',_'MOB') + x_('JAC',_'MOB')
 - x_('MOB',_'ATL') >= 70

Flow_Conservation_in_Node:JAC: - x_('JAC',_'ATL') - x_('JAC',_'MOB')
 - x_('JAC',_'RIC') >= -300

VARIABLES
x_('ATL',_'CO

In [5]:
# Optimize

prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])
print("Total Cost = %f" % value(prob.objective))

Status = Optimal
Total Cost = 22350.000000


In [6]:
# Print the value of the variables at the optimum
for l in links:
    if(x[l].varValue > 0):
        print("%s: %f" % (x[l].name, x[l].varValue))

x_('NEW',_'BOS'): 120.000000
x_('BOS',_'COL'): 20.000000
x_('ATL',_'COL'): 40.000000
x_('NEW',_'RIC'): 80.000000
x_('JAC',_'MOB'): 70.000000
x_('JAC',_'ATL'): 210.000000
